# Microtraining - Data-Analysis with Keras and Tensorflow

# Getting started with feed forward neural networks


This Training will show the basics step you will need to perform, to get started with Deep Learning on your dataset. The main idea is to give you a short introduction into the most important stages of data analysis with neural networks. The main stages are: 
* Data preprocessing
* Model setup
* Model training
* Model testing
* (Inference)
* Deployment

## Tensorflow and Keras

In this course we are using Tensorflow in combination with Keras as the main framework for our analysis. 

### Tensorflow 

<img src="https://upload.wikimedia.org/wikipedia/commons/1/11/TensorFlowLogo.svg">

[Tensorflow](https://www.tensorflow.org/) is a opensource framework provided and developed for machine and deep learning. The main purpose is to provide an easy and effective way to manipulate and calculate tensors.  

One of the key features of tensorflow is the ability to build computation graphs. These computation graphs can be build by using for example a High-Level language like Python. Afterwards these graphs are than initated and executed in C++.
The image below shows such a typical computation. A Input Tensor "flows" through" the graph.

<img src="https://www.tensorflow.org/images/tensors_flowing.gif">


### Keras

<img src="https://s3.amazonaws.com/keras.io/img/keras-logo-2018-large-1200.png" width="300">

[Keras](https://keras.io/) is a wrapper library which provides an easy and highly abstracted interface for low level frameworks like Tensorflow. Standard layers like Dense layers or CNN layers are already included in Keras making it the goto framework if you want to build fast prototype network. Of course the great flexibilty Tensorflow can provide gets lost, when you use Keras but it is possible to combine these two frameworks since Keras is basically just a wrapper for Tensorflow.   


In this workshop we will just import tensorflow since keras is part of tensorflow since Tensorflow 1.2.

In [1]:
#imports for the scripts
import tensorflow as tf

## Data preprocessing

<img src="http://tomba.co/analysis-consult/images/data-cleansing.jpg">

Data preprocessing is all about getting your data ready for your analysis. Normally this step takes about 80%-90% of the total time. 

When building neural networks you typically perform the following steps during this process:
* Loading your data
* Removing unwanted or corrupted data
* Matching your labels
* Shuffel
* Batch
* Split

In [2]:
# Load dataset
mnist = tf.keras.datasets.mnist

In [5]:
# Input Pipeline


(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0


In [6]:
# Inference

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

In [7]:
# Model compilation
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [8]:
# Training

model.fit(x_train, y_train, epochs=5)


Epoch 1/5
60000/60000 [==============================] - 20s 340us/step - loss: 0.2014 - acc: 0.9395
Epoch 2/5
60000/60000 [==============================] - 37s 618us/step - loss: 0.0819 - acc: 0.9749
Epoch 3/5
60000/60000 [==============================] - 37s 622us/step - loss: 0.0518 - acc: 0.9839
Epoch 4/5
60000/60000 [==============================] - 38s 633us/step - loss: 0.0367 - acc: 0.9883
Epoch 5/5
60000/60000 [==============================] - 36s 606us/step - loss: 0.0281 - acc: 0.9905


In [9]:
# Test

model.evaluate(x_test, y_test)

10000/10000 [==============================] - 1s 94us/step


[0.0693634459810739, 0.9797]

Todo:
* add more layers to your network and compare the accuracy before and after
* run your training phase with 10 instead of 3 epochs. Explain why the accuracy is dropping. Try adding a dropout layer afterwards
* change your train test ratio